In [165]:
import json
import os
import pandas as pd

# the folder location of .jsonl files you're compiling into dataframes
folder_path = 'C:/Users/linds/Downloads/wsd-data/wsd-data-main/wsd-hits'

# custom try statement used to access components in json data (whether or whether not it exists)
# input: a string of code
# output: the return value of the string of code (if works) or "n/a" (doesn't work)

def return_if_available(thing):
    try:
        return eval(thing)
    except:
        return "n/a"

In [167]:
# initialize all 
# initialize all 
main_columns = {"filename": ['file'], "main pitch event id":['eventId'], "main pitch result":['result'], "main pitch speed":['mph'], "main pitch spin":['rpm'], "main hit event id":['eventId'], "main hit speed":['mph'], "main hit spin":['rpm']}
event_columns = {"event angle start x":['x'], "event angle start y":['y'], "event type":['hit/no hit'], "event teamid":['mlbd'], "event personid":['mlbid'], "event eventid":['eventid']}
ball_columns = {"ball time start":['secs'], "ball pos start x":['feet'], "ball pos start y":['feet'], "ball pos start z":['feet'], "ball vel start x":['mph'], "ball vel start y":['mph'], "ball vel start z":['mph'], "ball acc start x":['mph/s'], "ball acc start y":['mph/s'], "ball acc start z":['mph/s'], "ball time end":['secs'], "ball pos end x":['feet'], "ball pos end y":['feet'], "ball pos end z":['feet'], "ball vel end x":['mph'], "ball vel end y":['mph'], "ball vel end z":['mph'], "ball acc end x":['mph/s'], "ball acc end y":['mph/s'], "ball acc end z":['mph/s']}
bat_columns = {"bat time start":['secs'], "bat pos head start x":['feet'], "bat pos head start y":['feet'], "bat pos head start z":['feet'], "bat pos handle start x":['feet'], "bat pos handle start y":['feet'], "bat pos handle start z":['feet'], "bat time end":['secs'], "bat pos head end x":['feet'], "bat pos head end y":['feet'], "bat pos head end z":['feet'], "bat pos handle end x":['feet'], "bat pos handle end y":['feet'], "bat pos handle end z":['feet']}

main_df = pd.DataFrame(main_columns)
event_df = pd.DataFrame(event_columns)
ball_df = pd.DataFrame(ball_columns)
bat_df = pd.DataFrame(bat_columns)

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [169]:
# dataset

for filename in os.listdir(folder_path):
    if filename.endswith('.jsonl'):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r') as json_file:
            data = json.load(json_file)
            # enter in data fields
            main_pitch_eventid = return_if_available("data['summary_acts']['pitch']['eventId']")
            main_pitch_result = return_if_available("data['summary_acts']['pitch']['result']")
            main_pitch_speed = return_if_available("data['summary_acts']['pitch']['speed']['mph']")
            main_pitch_spin = return_if_available("data['summary_acts']['pitch']['spin']['rpm']")
            main_hit_eventid = return_if_available("data['summary_acts']['hit']['eventId']")
            main_hit_speed = return_if_available("data['summary_acts']['hit']['speed']['mph']")
            main_hit_spin = return_if_available("data['summary_acts']['hit']['spin']['rpm']")
            row = {"filename": filename, "main pitch event id":main_pitch_eventid, "main pitch result":main_pitch_result, "main pitch speed":main_pitch_speed, "main pitch spin":main_pitch_spin, "main hit event id":main_hit_eventid, "main hit speed":main_hit_speed, "main hit spin":main_hit_spin}
            main_df.loc[len(main_df.index)] = row # add row to end of df

            event_angle_start_x = return_if_available("data['events'][0]['start']['angle'][0]")
            event_angle_start_y = return_if_available("data['events'][0]['start']['angle'][1]")
            event_type = return_if_available("data['events'][0]['type']")
            event_teamid = return_if_available("data['events'][0]['teamId']['mlbId']")
            event_personid = return_if_available("data['events'][0]['personId']['mlbId']")
            event_eventid = return_if_available("data['events'][0]['eventId']")
            row = {"event angle start x": event_angle_start_x, "event angle start y": event_angle_start_y, "event type": event_type, "event teamid":event_teamid, "event personid":event_personid, "event eventid":event_eventid}
            event_df.loc[len(event_df.index)] = row # add row to end of df

            ball_time_start = return_if_available("data['samples_ball'][0]['time']")
            ball_pos_start_x = return_if_available("data['samples_ball'][0]['pos'][0]")
            ball_pos_start_y = return_if_available("data['samples_ball'][0]['pos'][1]")
            ball_pos_start_z = return_if_available("data['samples_ball'][0]['pos'][2]")
            ball_vel_start_x = return_if_available("data['samples_ball'][0]['vel'][0]")
            ball_vel_start_y = return_if_available("data['samples_ball'][0]['vel'][1]")
            ball_vel_start_z = return_if_available("data['samples_ball'][0]['vel'][2]")
            ball_acc_start_x = return_if_available("data['samples_ball'][0]['acc'][0]")
            ball_acc_start_y = return_if_available("data['samples_ball'][0]['acc'][1]")
            ball_acc_start_z = return_if_available("data['samples_ball'][0]['acc'][2]")
            ball_time_end = return_if_available("data['samples_ball'][len(data['samples_ball']) - 1]['time']")
            ball_pos_end_x = return_if_available("data['samples_ball'][len(data['samples_ball']) - 1]['pos'][0]")
            ball_pos_end_y = return_if_available("data['samples_ball'][len(data['samples_ball']) - 1]['pos'][1]")
            ball_pos_end_z = return_if_available("data['samples_ball'][len(data['samples_ball']) - 1]['pos'][2]")
            ball_vel_end_x = return_if_available("data['samples_ball'][len(data['samples_ball']) - 1]['vel'][0]")
            ball_vel_end_y = return_if_available("data['samples_ball'][len(data['samples_ball']) - 1]['vel'][1]")
            ball_vel_end_z = return_if_available("data['samples_ball'][len(data['samples_ball']) - 1]['vel'][2]")
            ball_acc_end_x= return_if_available("data['samples_ball'][len(data['samples_ball']) - 1]['acc'][0]")
            ball_acc_end_y= return_if_available("data['samples_ball'][len(data['samples_ball']) - 1]['acc'][1]")
            ball_acc_end_z= return_if_available("data['samples_ball'][len(data['samples_ball']) - 1]['acc'][2]")
            row = {"ball time start":ball_time_start, "ball pos start x":ball_pos_start_x, "ball pos start y":ball_pos_start_y, "ball pos start z":ball_pos_start_z, "ball vel start x":ball_vel_start_x, "ball vel start y":ball_vel_start_y, "ball vel start z":ball_vel_start_z, "ball acc start x":ball_acc_start_x, "ball acc start y":ball_acc_start_y, "ball acc start z":ball_acc_start_z, "ball time end":ball_time_end, "ball pos end x":ball_pos_end_x, "ball pos end y":ball_pos_end_y, "ball pos end z":ball_pos_end_z, "ball vel end x":ball_vel_end_x, "ball vel end y":ball_vel_end_y, "ball vel end z":ball_vel_end_z, "ball acc end x":ball_acc_end_x, "ball acc end y":ball_acc_end_y, "ball acc end z":ball_acc_end_z}
            ball_df.loc[len(ball_df.index)] = row # add row to end of df

            bat_time_start = return_if_available("data['samples_bat'][0]['time']")
            bat_pos_head_start_x = return_if_available("data['samples_bat'][0]['head']['pos'][0]")
            bat_pos_head_start_y = return_if_available("data['samples_bat'][0]['head']['pos'][1]")
            bat_pos_head_start_z = return_if_available("data['samples_bat'][0]['head']['pos'][2]")
            bat_pos_handle_start_x = return_if_available("data['samples_bat'][0]['handle']['pos'][0]")
            bat_pos_handle_start_y = return_if_available("data['samples_bat'][0]['handle']['pos'][1]")
            bat_pos_handle_start_z = return_if_available("data['samples_bat'][0]['handle']['pos'][2]")
            bat_time_end = return_if_available("data['samples_bat'][len(data['samples_bat']) - 1]['time']")
            bat_pos_head_end_x = return_if_available("data['samples_bat'][len(data['samples_bat']) - 1]['head']['pos'][0]")
            bat_pos_head_end_y = return_if_available("data['samples_bat'][len(data['samples_bat']) - 1]['head']['pos'][1]")
            bat_pos_head_end_z = return_if_available("data['samples_bat'][len(data['samples_bat']) - 1]['head']['pos'][2]")
            bat_pos_handle_end_x = return_if_available("data['samples_bat'][len(data['samples_bat']) - 1]['handle']['pos'][0]")
            bat_pos_handle_end_y = return_if_available("data['samples_bat'][len(data['samples_bat']) - 1]['handle']['pos'][1]")
            bat_pos_handle_end_z = return_if_available("data['samples_bat'][len(data['samples_bat']) - 1]['handle']['pos'][2]")

            row = {"bat time start":bat_time_start, "bat pos head start x":bat_pos_head_start_x, "bat pos head start y":bat_pos_head_start_y, "bat pos head start z":bat_pos_head_start_z, "bat pos handle start x":bat_pos_handle_start_x, "bat pos handle start y":bat_pos_handle_start_y, "bat pos handle start z":bat_pos_handle_start_z, "bat time end":bat_time_end, "bat pos head end x":bat_pos_head_end_x, "bat pos head end y":bat_pos_head_end_y, "bat pos head end z":bat_pos_head_end_z, "bat pos handle end x":bat_pos_handle_end_x, "bat pos handle end y":bat_pos_handle_end_y, "bat pos handle end z":bat_pos_handle_end_z}
            bat_df.loc[len(bat_df.index)] = row # add row to end of df
            
            df = pd.concat([main_df, event_df, ball_df, bat_df], axis=1).reindex(main_df.index)



In [170]:
df

,filename,main pitch event id,main pitch result,main pitch speed,main pitch spin,main hit event id,main hit speed,main hit spin,event angle start x,event angle start y,event type,event teamid,event personid,event eventid,ball time start,ball pos start x,ball pos start y,ball pos start z,ball vel start x,ball vel start y,ball vel start z,ball acc start x,ball acc start y,ball acc start z,ball time end,ball pos end x,ball pos end y,ball pos end z,ball vel end x,ball vel end y,ball vel end z,ball acc end x,ball acc end y,ball acc end z,bat time start,bat pos head start x,bat pos head start y,bat pos head start z,bat pos handle start x,bat pos handle start y,bat pos handle start z,bat time end,bat pos head end x,bat pos head end y,bat pos head end z,bat pos handle end x,bat pos handle end y,bat pos handle end z
0,file,eventId,result,mph,rpm,eventId,mph,rpm,x,y,hit/no hit,mlbd,mlbid,eventid,secs,feet,feet,feet,mph,mph,mph,mph/s,mph/s,mph/s,secs,feet,feet,feet,mph,mph,mph,mph/s,mph/s,mph/s,secs,feet,feet,feet,feet,feet,feet,secs,feet,feet,feet,feet,feet,feet
1,12345634_10282.jsonl,ac29b4ab-63bc-4672-a29c-f4517fd03c85,HitIntoPlay,83.0,2350,daf9742c-869c-4370-9d66-59e217be1c89,24.0,1590,27.482589,65.83796,Hit,63813,797796542,daf9742c-869c-4370-9d66-59e217be1c89,0.00694,-2.596849,53.288877,3.383166,3.267184,-82.847555,4.581147,-10.855688,15.832688,-21.049011,19.087453,1.85256,67.832033,3.746256,0.1377,23.803842,-14.351191,-0.137172,-3.421289,-21.20476,-0.382787,-3.321382,-1.224789,6.246467,-1.762194,-2.465486,4.390434,0.846878,-1.503568,-1.096849,4.673711,-4.174917,-0.58695,4.812109
2,12345634_10962.jsonl,7139315c-faea-486f-94a4-c33e022866a9,Strike,97.0,2300,49f533e3-075e-428b-abc3-3303e0871a34,86.0,2640,-47.13422,29.57366,Hit,63813,412098649,49f533e3-075e-428b-abc3-3303e0871a34,0.015076,-2.062841,51.99718,5.335502,4.878706,-96.495549,-2.233093,-10.898125,21.615891,-9.702114,4.46758,0.067708,48.265008,7.081156,-0.73879,35.697612,-7.440813,0.070984,-5.601026,-19.996086,-0.448233,3.0862,-2.054622,7.060436,1.582198,-2.384678,4.814484,0.831608,2.770839,-1.583048,6.141535,4.736392,-0.06881,5.01951
3,12345634_11227.jsonl,444a6133-f51f-4fda-bef8-066259b9de2f,HitIntoPlay,97.0,2260,6851a681-5242-42ff-906c-125b455c7b08,89.0,2910,8.56738,36.176988,Hit,63813,797957728,6851a681-5242-42ff-906c-125b455c7b08,0.041829,-2.092848,48.239633,5.222792,3.829446,-95.622298,-3.400947,-7.812122,21.312156,-9.558819,13.140121,-2.561458,41.306717,5.200573,-1.099707,-33.062953,-17.367285,0.396357,3.545934,-21.295913,-0.444781,-4.230369,-0.181468,5.613366,-1.727313,-1.245015,5.476112,0.791679,-2.861363,-2.066628,5.613175,-4.412135,-0.215563,4.354635
4,12345634_11892.jsonl,bb53738f-62d0-4eb6-b9f9-0ebc5176f4a7,HitIntoPlay,87.0,2870,cac8582a-8ab9-4d75-809f-d0bbc309cb25,97.0,5100,-23.310262,-35.649607,Hit,63813,505414610,cac8582a-8ab9-4d75-809f-d0bbc309cb25,0.024473,-2.876076,51.027176,4.990343,4.373763,-86.360001,-0.230943,2.313181,15.196678,-22.951512,9.074988,7.41016,51.548514,3.872795,-29.721641,-10.111926,-11.460872,3.981848,1.330719,-21.445486,-0.402128,-3.572167,-0.922298,6.384278,-1.598487,-1.886854,4.775092,0.844334,-2.282921,-1.609688,5.304694,-3.241453,0.39078,3.725263
5,12345634_12936.jsonl,e866814f-8f0c-45e4-a9f1-f5ea9354b6d7,Strike,94.0,2240,04d236e0-7a67-4c2e-af11-d8877bf73219,86.0,3950,83.529292,48.865278,Hit,63813,849653732,04d236e0-7a67-4c2e-af11-d8877bf73219,0.044344,-1.719649,48.095764,5.48259,7.612508,-92.426992,-2.735903,-14.30713,17.882604,-15.465568,3.804043,1.706542,21.749272,10.367554,-0.28443,34.954278,3.019046,0.035709,-5.187269,-22.501712,-0.435464,-3.595469,-0.81591,6.138747,-1.538842,-1.902264,4.722633,0.814435,-3.549686,-0.470943,6.663956,-2.791968,1.115527,4.584476
6,12345634_1574.jsonl,f26b42be-568b-420a-8032-b9f705155507,HitIntoPlay,95.0,2250,b5ae0716-5252-42a9-a0e4-043fc79c3ef4,90.0,2290,16.612458,29.022325,Hit,63813,797957728,b5ae0716-5252-42a9-a0e4-043fc79c3ef4,0.04005,-1.286679,48.617182,5.275065,3.010985,-93.443675,-4.32992,-10.80

In [173]:
# clean up data / make data easily processable

df_processed = df.copy()
# delete the first row (irrelevant data)
df_processed = df_processed.iloc[1:]

# convert relevant non-numeric data with numeric data
df_processed.drop(['filename', 'main pitch event id', 'main hit event id', 'event personid', 'event eventid'], axis=1, inplace=True)
df_processed['main pitch result'] = df_processed['main pitch result'].map({'HitIntoPlay': 1, 'Strike': 0}) # for main pitch result, all strike = 0, hitintoplay = 1
df_processed['event type'] = df_processed['event type'].map({'Hit': 1, 'n/a': 0}) # for event type, all hit = 1, n/a = 0

# drop all columns with non-numeric values
for col in df_processed.columns:
    df_processed = df_processed[pd.to_numeric(df_processed[col], errors='coerce').notnull()]

In [175]:
# display everything

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(df_processed)

,main pitch result,main pitch speed,main pitch spin,main hit speed,main hit spin,event angle start x,event angle start y,event type,event teamid,ball time start,ball pos start x,ball pos start y,ball pos start z,ball vel start x,ball vel start y,ball vel start z,ball acc start x,ball acc start y,ball acc start z,ball time end,ball pos end x,ball pos end y,ball pos end z,ball vel end x,ball vel end y,ball vel end z,ball acc end x,ball acc end y,ball acc end z,bat time start,bat pos head start x,bat pos head start y,bat pos head start z,bat pos handle start x,bat pos handle start y,bat pos handle start z,bat time end,bat pos head end x,bat pos head end y,bat pos head end z,bat pos handle end x,bat pos handle end y,bat pos handle end z
1,1.0,83.0,2350,24.0,1590,27.482589,65.83796,1.0,63813,0.00694,-2.596849,53.288877,3.383166,3.267184,-82.847555,4.581147,-10.855688,15.832688,-21.049011,19.087453,1.85256,67.832033,3.746256,0.1377,23.803842,-14.351191,-0.137172,-3.421289,-21.20476,-0.382787,-3.321382,-1.224789,6.246467,-1.762194,-2.465486,4.390434,0.846878,-1.503568,-1.096849,4.673711,-4.174917,-0.58695,4.812109
2,0.0,97.0,2300,86.0,2640,-47.13422,29.57366,1.0,63813,0.015076,-2.062841,51.99718,5.335502,4.878706,-96.495549,-2.233093,-10.898125,21.615891,-9.702114,4.46758,0.067708,48.265008,7.081156,-0.73879,35.697612,-7.440813,0.070984,-5.601026,-19.996086,-0.448233,3.0862,-2.054622,7.060436,1.582198,-2.384678,4.814484,0.831608,2.770839,-1.583048,6.141535,4.736392,-0.06881,5.01951
3,1.0,97.0,2260,89.0,2910,8.56738,36.176988,1.0,63813,0.041829,-2.092848,48.239633,5.222792,3.829446,-95.622298,-3.400947,-7.812122,21.312156,-9.558819,13.140121,-2.561458,41.306717,5.200573,-1.099707,-33.062953,-17.367285,0.396357,3.545934,-21.295913,-0.444781,-4.230369,-0.181468,5.613366,-1.727313,-1.245015,5.476112,0.791679,-2.861363,-2.066628,5.613175,-4.412135,-0.215563,4.354635
4,1.0,87.0,2870,97.0,5100,-23.310262,-35.649607,1.0,63813,0.024473,-2.876076,51.027176,4.990343,4.373763,-86.360001,-0.230943,2.313181,15.196678,-22.951512,9.074988,7.41016,51.548514,3.872795,-29.721641,-10.111926,-11.460872,3.981848,1.330719,-21.445486,-0.402128,-3.572167,-0.922298,6.384278,-1.598487,-1.886854,4.775092,0.844334,-2.282921,-1.609688,5.304694,-3.241453,0.39078,3.725263
5,0.0,94.0,2240,86.0,3950,83.529292,48.865278,1.0,63813,0.044344,-1.719649,48.095764,5.48259,7.612508,-92.426992,-2.735903,-14.30713,17.882604,-15.465568,3.804043,1.706542,21.749272,10.367554,-0.28443,34.954278,3.019046,0.035709,-5.187269,-22.501712,-0.435464,-3.595469,-0.81591,6.138747,-1.538842,-1.902264,4.722633,0.814435,-3.549686,-0.470943,6.663956,-2.791968,1.115527,4.584476
6,1.0,95.0,2250,90.0,2290,16.612458,29.022325,1.0,63813,0.04005,-1.286679,48.617182,5.275065,3.010985,-93.443675,-4.32992,-10.809683,24.289477,-8.08009,8.59849,-59.597212,68.259011,0.834132,-26.117042,-16.555158,-5.417106,3.544204,2.216284,-22.744168,-0.436415,-4.337222,-0.321549,5.872455,-1.83654,-1.290348,5.399936,0.806829,-1.582852,-2.170901,4.30643,-3.987001,-0.935864,4.638107
7,1.0,90.0,2330,82.0,5780,24.836491,69.823967,1.0,90068,0.019891,-1.498238,51.529816,5.598103,3.464426,-89.025991,-2.505826,-9.76155,20.089941,-10.484596,6.399657,73.092124,141.339759,-0.043701,6.142703,11.598134,-57.710173,-1.135257,-2.143496,-13.084655,-0.40991,-3.277695,-0.585838,6.82112,-1.626526,-1.977011,5.161742,0.809999,-1.233641,-1.915929,3.522765,-3.416171,-1.342213,5.046796
8,0.0,93.0,2180,69.0,3080,88.188223,47.522762,1.0,90068,0.01219,-1.936452,52.471294,5.19124,4.654131,-92.883862,-3.151139,-7.273018,18.687306,-11.704065,5.574335,1.703569,49.944968,4.112246,0.295122,27.999224,-14.104577,0.419493,-4.667883,-20.944637,-0.434233,-4.130509,-0.844832,4.457198,-1.411974,-0.921757,4.594696,0.81221,-1.778627,-2.269247,4.16081,-3.696675,-0.363938,4.486604
9,1.0,95.0,3350,104.0,570,-13.436606,9.382088,1.0,63813,0.01968,-1.144677,51.397075,5.473864,4.47236,-94.615385,-4.197993,-12.090698,21.709611,-7.386435,18.756122,34.831048,6.771882,0.0329

In [177]:
# making correlation table

corr_matrix = df_processed.corr(numeric_only = False)

# styling correlation df
def cond_formatting(x):
    if x > 0.5:
        return 'background-color: lightgreen'
    else:
        return None
    
#display DataFrame with conditional formatting applied    
corr_matrix = corr_matrix.style.map(cond_formatting)
display(corr_matrix)

,main pitch result,main pitch speed,main pitch spin,main hit speed,main hit spin,event angle start x,event angle start y,event type,event teamid,ball time start,ball pos start x,ball pos start y,ball pos start z,ball vel start x,ball vel start y,ball vel start z,ball acc start x,ball acc start y,ball acc start z,ball time end,ball pos end x,ball pos end y,ball pos end z,ball vel end x,ball vel end y,ball vel end z,ball acc end x,ball acc end y,ball acc end z,bat time start,bat pos head start x,bat pos head start y,bat pos head start z,bat pos handle start x,bat pos handle start y,bat pos handle start z,bat time end,bat pos head end x,bat pos head end y,bat pos head end z,bat pos handle end x,bat pos handle end y,bat pos handle end z
main pitch result,1.000000,-0.128975,-0.017325,0.451164,-0.311701,-0.267365,-0.501556,nan,-0.039550,0.068414,0.023615,-0.052615,-0.141039,-0.101769,0.137257,0.017087,0.016756,-0.099518,-0.157895,0.672874,-0.166024,0.130510,-0.279497,0.033574,-0.764147,0.034253,-0.042527,0.713081,-0.113383,0.126418,-0.031134,0.009859,0.053385,-0.043995,-0.079291,0.031914,0.134161,-0.089083,0.081594,0.125690,-0.064669,0.195933,-0.328168
main pitch speed,-0.128975,1.000000,-0.068025,0.082041,-0.100317,-0.028815,-0.037893,nan,-0.037789,0.107316,0.041409,-0.190473,0.211367,0.220885,-0.997412,-0.761637,-0.450903,0.828224,0.820178,-0.202581,-0.069692,0.004673,0.122601,0.079609,0.065249,0.037412,-0.103648,-0.024934,0.021870,-0.990058,0.089432,0.096332,0.045980,0.126087,0.145611,0.144846,-0.736805,0.129553,-0.173277,-0.015698,0.123249,-0.064594,0.064275
main pitch spin,-0.017325,-0.068025,1.000000,-0.066991,-0.043038,0.011342,-0.099947,nan,-0.122765,-0.000492,-0.067751,0.011282,0.057034,0.031880,0.064520,0.086836,0.082078,-0.075070,-0.105819,0.044359,-0.069452,-0.124113,0.027671,-0.045310,0.028911,0.088393,0.013182,-0.048980,-0.159556,0.064894,-0.039691,0.006404,-0.028204,-0.006554,-0.023903,0.094274,0.107873,0.017979,0.020028,-0.050910,-0.014664,-0.001403,-0.094937
main hit speed,0.451164,0.082041,-0.066991,1.000000,-0.151060,-0.189358,-0.329394,nan,-0.092035,0.028885,0.156705,-0.031921,0.140014,-0.135091,-0.080170,-0.216645,0.068049,0.083287,0.057653,0.234228,-0.132869,0.235415,-0.227971,0.026102,-0.412965,0.006865,-0.109109,0.398091,0.009613,-0.081210,0.047166,-0.006183,0.004872,0.049874,-0.093660,0.039376,-0.036957,-0.102180,-0.021363,0.222391,0.022185,0.210123,-0.131064
main hit spin,-0.311701,-0.100317,-0.043038,-0.151060,1.000000,-0.007385,0.341894,nan,0.046246,-0.004813,-0.108939,0.008472,0.069870,0.088681,0.101851,0.043967,-0.007793,-0.027541,-0.021281,-0.273807,0.154065,0.084511,0.070539,-0.004976,0.324819,-0.259904,0.019461,-0.260206,0.232256,0.113179,-0.043431,0.029157,-0.023471,-0.031812,-0.075635,0.113154,-0.072037,0.039134,-0.202370,-0.140590,-0.044547,-0.208208,0.139291
event angle start x,-0.267365,-0.028815,0.011342,-0.189358,-0.007385,1.000000,0.201677,nan,0.151439,-0.193809,0.080635,0.201165,0.138728,-0.108647,0.023162,0.043932,0.110167,0.014322,-0.112137,-0.247731,0.076276,-0.006271,0.071137,-0.050223,0.204188,-0.025139,0.113996,-0.192823,0.100794,0.030035,-0.493945,-0.108574,-0.170833,-0.485422,0.061984,-0.065310,0.068206,-0.336098,-0.040794,0.103595,-0.468025,0.018927,0.245870
event angle start y,-0.501556,-0.037893,-0.099947,-0.329394,0.341894,0.201677,1.000000,nan,0.170598,-0.071451,-0.082264,0.067318,0.057635,0.054899,0.034540,0.103415,0.021910,0.051437,0.120158,-0.399484,0.179997,0.100371,0.058737,0.029088,0.414747,-0.238878,0.011305,-0.373224,0.287456,0.049332,-0.039072,0.140419,-0.134308,-0.029600,0.156926,0.052373,-0.135383,-0.007718,-0.162692,-0.083457,-0.012373,-0.264312,0.423863
event type,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
event teamid,-0.039550,-0.037789,-0.122765,-0.092035,0.046246,0.151439,0.170598,nan,1.000000,-0.192182,-0.013661,0.192785,0.172021,0.10

In [155]:
# download styled dataframe as an html file

download_location = "C:/Users/linds/Downloads/"

corr_html = corr_matrix.to_html()
file = open(download_location + "corr_styled_df.html", "w")
file.write(corr_html)
file.close()
# corr_matrix.to_csv(r'' + download_location + 'main_df.csv')